In [2]:
import os
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4"

In [8]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv('OPEN_AI_KEY'),
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [5]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [13]:
functions = [
    {
        "name": "get_method_name",
        "description": "Get a method name",
        "parameters": {
            "type": "object",
            "properties": {
                "method_content": {
                    "type": "string",
                    "description": "The method code",
                }
            },
            "required": ["method_content"],
        },
    }
]

In [14]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is a good candidate for the java method?"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'id': 'chatcmpl-7diIvrlYWporejoXbCncnxp3ZbR9V',
 'object': 'chat.completion',
 'created': 1689699453,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "I'm sorry for the confusion, but could you please provide more details? Are you asking about a specific type of Java method or do you want a general Java method example? Your question is a bit ambiguous, and additional details would help to give a more accurate response."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 83, 'completion_tokens': 55, 'total_tokens': 138}}

In [16]:
messages.append({"role": "user", "content": ''' 
/** 
 * Compile all of the compilable panes that're opened.
 * @param message if it should send a message saying it's compiled sucessfully.
 * @return true if no errors, false if it failed to compile.
 */
public static boolean compile(boolean message,boolean successAlert){
  BytecodeViewer.updateBusyStatus(true);
  boolean noErrors=true;
  boolean actuallyTried=false;
  for (  java.awt.Component c : BytecodeViewer.viewer.workPane.getLoadedViewers()) {
    if (c instanceof ClassViewer) {
      ClassViewer cv=(ClassViewer)c;
      if (noErrors && !cv.bytecodeViewPanel1.compile())       noErrors=false;
      if (noErrors && !cv.bytecodeViewPanel2.compile())       noErrors=false;
      if (noErrors && !cv.bytecodeViewPanel3.compile())       noErrors=false;
      actuallyTried=compile_extraction_2(actuallyTried,cv);
      if (cv.bytecodeViewPanel3.textArea != null && cv.bytecodeViewPanel3.textArea.isEditable())       actuallyTried=true;
    }
  }
  if (message) {
    if (actuallyTried) {
      if (noErrors && successAlert)       BytecodeViewer.showMessage("Compiled Successfully.");
    }
 else {
      BytecodeViewer.showMessage("You have no editable panes opened, make one editable and try again.");
    }
  }
  BytecodeViewer.updateBusyStatus(false);
  return true;
}'''})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_method_name',
  'arguments': '{\n"method_content": "/** \\n* Compile all of the compilable panes that\'re opened.\\n* @param message if it should send a message saying it\'s compiled sucessfully.\\n* @return true if no errors, false if it failed to compile.\\n*/\\npublic static boolean compile(boolean message,boolean successAlert){\\nBytecodeViewer.updateBusyStatus(true);\\nboolean noErrors=true;\\nboolean actuallyTried=false;\\nfor ( java.awt.Component c : BytecodeViewer.viewer.workPane.getLoadedViewers()) {\\nif (c instanceof ClassViewer) {\\nClassViewer cv=(ClassViewer)c;\\nif (noErrors && !cv.bytecodeViewPanel1.compile()) noErrors=false;\\nif (noErrors && !cv.bytecodeViewPanel2.compile()) noErrors=false;\\nif (noErrors && !cv.bytecodeViewPanel3.compile()) noErrors=false;\\nactuallyTried=compile_extraction_2(actuallyTried,cv);\\nif (cv.bytecodeViewPanel3.textArea != null && cv.bytecodeViewPanel3.textArea.isEdita

In [17]:
chat_response.json()

{'id': 'chatcmpl-7diKTl1hFvcC5c5mi7FxwDEvdvb8e',
 'object': 'chat.completion',
 'created': 1689699549,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_method_name',
     'arguments': '{\n"method_content": "/** \\n* Compile all of the compilable panes that\'re opened.\\n* @param message if it should send a message saying it\'s compiled sucessfully.\\n* @return true if no errors, false if it failed to compile.\\n*/\\npublic static boolean compile(boolean message,boolean successAlert){\\nBytecodeViewer.updateBusyStatus(true);\\nboolean noErrors=true;\\nboolean actuallyTried=false;\\nfor ( java.awt.Component c : BytecodeViewer.viewer.workPane.getLoadedViewers()) {\\nif (c instanceof ClassViewer) {\\nClassViewer cv=(ClassViewer)c;\\nif (noErrors && !cv.bytecodeViewPanel1.compile()) noErrors=false;\\nif (noErrors && !cv.bytecodeViewPanel2.compile()) noErrors=false;\\nif (noErrors && !cv.bytecodeView